# Directionaly of the adaptive response

We will use other methods to evaluate the phenotypic consequences of our experiment.


In [91]:
#Libraries
library('tidyverse')
library('rsample')
library(broom)
library(purrr)

#Functions
#Get the centroids
grammian = function(data606){
    
centroids=data606 %>% select(clone,Condition,PC1,PC2,PC3,PC4) %>% 
filter(Condition==37) %>% group_by(clone) %>%  summarize_at(c("PC1", "PC2","PC3","PC4"), mean, na.rm = TRUE)
centroids$condition=37
centroids43=data606 %>% select(clone,Condition,PC1,PC2,PC3,PC4) %>% 
filter(Condition==43) %>% group_by(clone) %>%  summarize_at(c("PC1", "PC2","PC3","PC4"), mean, na.rm = TRUE)
centroids43$condition=43
centroids=rbind(centroids,centroids43)
centroids15=data606 %>% select(clone,Condition,PC1,PC2,PC3,PC4) %>% 
filter(Condition==15) %>% group_by(clone) %>%  summarize_at(c("PC1", "PC2","PC3","PC4"), mean, na.rm = TRUE)
centroids15$condition=15
centroids=rbind(centroids,centroids15)

#Gramiam
strains=levels(centroids$clone)
gram=data.frame(clone=strains)
for  (i in 1:length(strains) ){
    soca=as.character(strains[i])
    t=centroids %>% filter(clone==soca) %>% select(PC1,PC2,PC3,PC4)
    mat=t(as.matrix(t))
    gram$gramian[i]=det(t(mat) %*% mat)
    }
return(gram)
}

In [3]:
#Read the PCA transformmed data
data606=read.csv('606_scores.csv')
data607=read.csv('607_scores.csv')
strain_names=read.csv('strain_names.csv')
row.names(strain_names)=strain_names$nickname
## Format columns
data606$Condition=as.factor(data606$Condition)
data606$Treatment=as.factor(data606$Treatment)
data606$Strain=as.factor(data606$Strain)
data607$Condition=as.factor(data607$Condition)
data607$Treatment=as.factor(data607$Treatment)
data607$Strain=as.factor(data607$Strain)

## Grammian

In [110]:
#Bootstrap

boots <- bootstraps(data606, times = 10)

head(as.data.frame((boots$splits[1])))


Warning message:
“`optional` is kept for consistency with the underlying class but TRUE values will be ignored.”

,clone,Replicate,Strain,Treatment,Evolved,Condition,PC1,PC2,PC3,PC4,⋯,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91
56,Lyanna,Lyanna.15.2,606,slow,Evolved,15,-0.7734250,0.05747144,0.09795112,-0.0007775446,⋯,-0.0043386671,-0.0069366510,4.165754e-03,0.0039545555,-0.0012470878,0.0003686750,0.0007006693,-0.0008874835,-2.041892e-03,-0.0005578750
25,Baelish,Baelish.43.1,606,slow,Evolved,43,1.0640881,-0.23263560,-0.13191340,0.1950026307,⋯,-0.0019888934,0.0022548129,-1.649298e-03,0.0013089042,-0.0079317252,-0.0042788261,0.0040694887,-0.0012789045,-7.719790e-04,-0.0026648398
75,Ramsay,Ramsay.15.3,606,slow,Evolved,15,-1.0612246,-0.24587086,0.07055231,-0.0932722747,⋯,0.0020916615,0.0041296716,-4.060216e-03,-0.0119170700,-0.0009572987,-0.0028197275,0.0008265956,0.0015682055,-7.432402e-04,-0.0004472093
126,Thoros,Thoros.43.3,606,random,Evolved,43,0.7036399,-0.40662107,-0.10823590,-0.0425285648,⋯,-0.0009695023,0.0087268954,7.071397e-05,-0.0021845038,-0.0022215561,-0.0008312613,-0.0026361777,-0.0026043646,-2.639333e-03,-0.0005547585
26,Baelish,Baelish.43.2,606,slow,Evolved,43,1.1538218,-0.05694013,-0.57123684,0.3022228439,⋯,0.0094650962,-0.0002014745,-1.818772e-03,0.0006284329,0.0052907520,0.0065610533,-0.0060586931,-0.0026255478,2.672284e-03,0.0017944798
89,Renly,Renly.43.2,606,slow,Evolved,43,1.0303158,-0.20927008,-0.55230682,-0.0546141377,⋯,0.0024835907,0.0017889077,-5.821384e-03,-0.0014101578,-0.0010777104,-0.0001783357,0.0028229729,0.0007325533,-6.513061e-05,0.0003485818


clone,gramian
606p,0.11728066
Baelish,0.28091994
Dayne,0.54082507
Egg,0.69649446
Khaleesi,0.01630194
Lyanna,0.13124774
Nymeria,0.40799767
Ramsay,0.08921372
Renly,0.01499908
Rhaegal,0.06375882


## Get the comparisons

In [142]:
#comparisons
data = data607
T_ref = 37
T_test = 15
refname = "607p"
comp_file=compare_to_ancestral(data, T_ref, T_test ,refname)
file_name=paste(refname,T_test,"vs",T_ref,".csv",sep= "_")
write.csv(comp_file,file_name)

### Correction of type I error and directionality assessment

In [145]:
#To compare
T_ref = 43
T_test = 43

#Load the data and the reference set
data_name=paste(refname,T_test,"vs",T_ref,".csv",sep= "_")
data=read.csv(data_name)
alpha=0.05
control_name=paste(refname,T_test,"vs","37",".csv",sep= "_")
control=read.csv(control_name)
#Rename Columns
colnames(control)[4]="pvalref"
colnames(data)[4]="pvaltest"
data$logi_test=as.character(data$logi_test)
control$logi_test=as.character(control$logi_test)
#Multiple comparison correction
control$pvalref=p.adjust(control$pvalref,method="fdr")
data$pvaltest=p.adjust(data$pvaltest,method="fdr")
data$logi_test[data$pvaltest>=alpha] = 'equal'
control$logi_test[control$pvalref>=alpha] = 'equal'

#Add PCdir to data
references = control %>% filter((control$clone==refname))
row.names(references)=references$PC
data=data %>%  mutate(PCdir=references[as.character(PC),'logi_test'])

#filter controls
data = data %>% filter(!(data$clone==refname))
control = control %>% filter(!(control$clone==refname))
data$comparison=""

# Test Classifyier
for (i in 1:nrow(data)) {

	if (control$logi_test[i]=="less" && data$logi_test[i]=="greater" && data$PCdir[i]=="less") {
		data$comparison[i]="Partially Restored"} 
	if(control$logi_test[i]=="greater" && data$logi_test[i]=="less"  && data$PCdir[i]=="greater") {
		data$comparison[i]="Partially Restored"}
	if(control$logi_test[i]=="less" && data$logi_test[i]=="less" && data$PCdir[i]=="less") {
		data$comparison[i]="Reinforced"}
	if(control$logi_test[i]=="greater" && data$logi_test[i]=="greater" && data$PCdir[i]=="greater") {
		data$comparison[i]="Reinforced"}
	if(control$logi_test[i]=="less" && data$logi_test[i]=="less" && data$PCdir[i]=="greater") {
		data$comparison[i]="Over-restored"}
	if(control$logi_test[i]=="greater" && data$logi_test[i]=="greater" && data$PCdir[i]=="less") {
		data$comparison[i]="Over-restored"}
	if(control$logi_test[i]=="greater" && data$logi_test[i]=="equal" && data$PCdir[i]=="greater") {
		data$comparison[i]="Unrestored"}
	if(control$logi_test[i]=="less" && data$logi_test[i]=="equal"&& data$PCdir[i]=="less") {
		data$comparison[i]="Unrestored"}
	if(control$logi_test[i]=="equal" && data$logi_test[i]=="greater"  && data$PCdir[i]=="less") {
		data$comparison[i]="Restored"}
	if(control$logi_test[i]=="equal" && data$logi_test[i]=="less"  && data$PCdir[i]=="greater") {
		data$comparison[i]="Restored"}
	if(control$logi_test[i]=="less" && data$logi_test[i]=="less"  && data$PCdir[i]=="equal") {
		data$comparison[i]="Novel"}
	if(control$logi_test[i]=="greater" && data$logi_test[i]=="greater"  && data$PCdir[i]=="equal") {
		data$comparison[i]="Novel"}
    if(control$logi_test[i]=="less" && data$logi_test[i]=="equal"  && data$PCdir[i]=="equal") {
		data$comparison[i]="Uninformative"}
    if(control$logi_test[i]=="greater" && data$logi_test[i]=="equal"  && data$PCdir[i]=="equal") {
		data$comparison[i]="Uninformative"}
	if(control$logi_test[i]=="equal" && data$logi_test[i]=="equal"  && data$PCdir[i]=="equal") {
		data$comparison[i]="Uninformative"}
    if(control$logi_test[i]=="equal" && data$logi_test[i]=="equal"  && data$PCdir[i]=="less") {
		data$comparison[i]="Uninformative"}
    if(control$logi_test[i]=="equal" && data$logi_test[i]=="equal"  && data$PCdir[i]=="greater") {
		data$comparison[i]="Uninformative"}
	}

#Save
file_name=paste(refname,T_test,"directionality.csv",sep= "_")
write.csv(data,file_name)

#Spread version
spreado = data %>% select(clone,PC,comparison) %>% spread(PC,comparison)
spreado=spreado %>%  mutate(names=strain_names[as.character(clone),'name'])
file_name=paste(refname,T_test,"directionality_spread.csv",sep= "_")
write.csv(spreado,file_name)